In [1]:
import flair
flair.set_seed(2)

from flair.data import Corpus, Sentence
from flair.datasets import TREC_6, CSVClassificationCorpus
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

import torch
import argparse
import json
import csv
import re
import time

import pandas as pd

In [2]:
learning_rate = 5.0e-5
mini_batch_size = 16
max_epochs = 8
model_name = "sergeyzh/rubert-tiny-turbo"

In [3]:

        
column_name_map = {0: "text", 1: "label"}
corpus: Corpus = CSVClassificationCorpus("flair_data",
                                            column_name_map,
                                            skip_header=False,
                                            delimiter='\t',    # tab-separated files
                                            label_type='label')



label_dict = corpus.make_label_dictionary(label_type='label')

2024-11-16 23:02:49,738 Reading data from flair_data
2024-11-16 23:02:49,746 Train: flair_data/train.csv
2024-11-16 23:02:49,751 Dev: flair_data/dev.csv
2024-11-16 23:02:49,755 Test: flair_data/test.csv
2024-11-16 23:02:50,057 Computing label dictionary. Progress:


0it [00:00, ?it/s]
4068it [00:33, 121.38it/s]

2024-11-16 23:03:23,953 Dictionary created for label 'label' with 9 values: FOOD_GOODS (seen 922 times), NON_FOOD_GOODS (seen 895 times), SERVICE (seen 884 times), LEASING (seen 380 times), LOAN (seen 380 times), REALE_STATE (seen 256 times), BANK_SERVICE (seen 215 times), NOT_CLASSIFIED (seen 117 times), TAX (seen 19 times)


In [4]:
document_embeddings = TransformerDocumentEmbeddings(model_name, fine_tune=True)
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type='label')

trainer = ModelTrainer(classifier, corpus)

trainer.fine_tune(f'./models/rubert_{str(time.time()).split(".")[0]}',
                    learning_rate=learning_rate,
                    mini_batch_size=mini_batch_size,
                    max_epochs=max_epochs,
                    monitor_test=True, train_with_dev=True, save_model_each_k_epochs=1)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

...

2024-11-16 23:09:22,855 TEST : loss 0.016288593411445618 - f1-score (micro avg)  0.996


2024-11-16 23:09:23,270 ----------------------------------------------------------------------------------------------------
2024-11-16 23:09:23,272 Testing using last state of model ...


100%|██████████| 32/32 [00:00<00:00, 62.98it/s]

2024-11-16 23:09:23,812 
Results:
- F-score (micro) 0.996
- F-score (macro) 0.9976
- Accuracy 0.996

By class:
                precision    recall  f1-score   support

NON_FOOD_GOODS     1.0000    0.9792    0.9895        96
    FOOD_GOODS     0.9890    1.0000    0.9945        90
       SERVICE     0.9888    1.0000    0.9944        88
  BANK_SERVICE     1.0000    1.0000    1.0000        49
           TAX     1.0000    1.0000    1.0000        48
          LOAN     1.0000    1.0000    1.0000        41
       LEASING     1.0000    1.0000    1.0000        38
   REALE_STATE     1.0000    1.0000    1.0000        27
NOT_CLASSIFIED     1.0000    1.0000    1.0000        23

      accuracy                         0.9960       500
     macro avg     0.9975    0.9977    0.9976       500
  weighted avg     0.9960    0.9960    0.9960       500

2024-11-16 23:09:23,813 ----------------------------------------------------------------------------------------------------


{'test_score': 0.996}

In [5]:
sentences = [sentence for sentence in corpus.test]
real_tags = [sentence.tag for sentence in sentences]
predicted_tags = []
for i in range(0, len(sentences), 8):
    sents = sentences[i:i+8]
    classifier.predict(sents, mini_batch_size=8)
    for sent in sents:
        predicted_tags.append(sent.tag)

In [6]:
for index in range(len(predicted_tags)):
    if predicted_tags[index] != real_tags[index]:
        print(corpus.test[index].text)
        print(f"| Real tag: | {real_tags[index]} | Predicted tag: | {predicted_tags[index]} |\n")

Оплата за Уход за одеждой и обувью по счету 11837472833255495630 от 21.06.2024г. Сумма 4110-00
| Real tag: | NON_FOOD_GOODS | Predicted tag: | SERVICE |

Оплата за Крем Бархатные ручки Питательный для рук с маслом ши 80мл по счету 97745000424439727136 от 28 января 2023 Сумма 4890,00
| Real tag: | NON_FOOD_GOODS | Predicted tag: | FOOD_GOODS |

